In [1]:
import sys
sys.path.append(r"C:\Users\Reem Darawsheh\Desktop\PythonProject/PythonProject/scripts")  # المسار المناسب لمكان السكريبت

from hybrid_representation import build_hybrid_representation_in_chunks_joblib

# مثال على الاستدعاء
build_hybrid_representation_in_chunks_joblib(
    tfidf_path="antique/train/doc/tfidf.joblib",
    bert_path="antique/train/doc/bert_embedding.joblib",
    save_path="antique/train/doc/hybrid_chunks"
)


In [7]:

build_hybrid_representation_in_chunks_joblib(
    tfidf_path=r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Data Representation\TF-IDF\beir\quora\test\doc\tfidf_data.joblib",
    bert_path=r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Data Representation\Bert\beir\quora\test\doc\bert_embedding.joblib",
    save_path=r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Data Representation\Hybrid\beir\quora\test\chunks",
    chunk_size=5000)

🚀 بناء التمثيل الهجين باستخدام TF-IDF وBERT بشكل دفعات (Chunks)...


🧩 Processing Chunks: 100%|██████████| 105/105 [01:30<00:00,  1.16it/s]


✅ تم حفظ جميع التمثيلات الهجينة بصيغة joblib في: C:\Users\Azzam\PycharmProjects\PythonProject\Data Representation\Hybrid\beir\quora\test\chunks


In [5]:
import os
import joblib
import numpy as np
from tqdm import tqdm


def build_clean_hybrid_representation_in_chunks(
    tfidf_path,
    bert_path,
    save_path,
    chunk_size=5000
):
    print("🚀 بدء معالجة التمثيلات الهجينة مع التحقق من التداخل...")

    # 1. تحميل البيانات
    tfidf_data = joblib.load(tfidf_path)
    bert_data = joblib.load(bert_path)

    tfidf_matrix = tfidf_data["tfidf_matrix"]
    tfidf_doc_ids = tfidf_data["doc_ids"]

    bert_matrix = np.array(bert_data["embeddings_matrix"], dtype=np.float32)
    bert_doc_ids = bert_data["doc_ids"]

    # 2. إيجاد التقاطع بين doc_ids
    common_doc_ids = list(set(tfidf_doc_ids) & set(bert_doc_ids))
    common_doc_ids.sort()  # ترتيب ثابت

    print(f"📌 عدد الوثائق المشتركة: {len(common_doc_ids)} / TF-IDF: {len(tfidf_doc_ids)} / BERT: {len(bert_doc_ids)}")

    # 3. بناء map من doc_id إلى index لكلا المصدرين
    tfidf_id_to_index = {doc_id: idx for idx, doc_id in enumerate(tfidf_doc_ids)}
    bert_id_to_index = {doc_id: idx for idx, doc_id in enumerate(bert_doc_ids)}

    # 4. تصفية وتمثيل البيانات المشتركة فقط
    filtered_tfidf = tfidf_matrix[[tfidf_id_to_index[doc_id] for doc_id in common_doc_ids]]
    filtered_bert = np.array([bert_matrix[bert_id_to_index[doc_id]] for doc_id in common_doc_ids], dtype=np.float32)

    # 5. التأكد
    assert filtered_tfidf.shape[0] == filtered_bert.shape[0] == len(common_doc_ids)

    os.makedirs(save_path, exist_ok=True)
    total_docs = len(common_doc_ids)

    # 6. التقطيع إلى Chunks
    for start in tqdm(range(0, total_docs, chunk_size), desc="🧩 معالجة Chunks"):
        end = min(start + chunk_size, total_docs)

        tfidf_chunk = filtered_tfidf[start:end]
        bert_chunk = filtered_bert[start:end]
        chunk_doc_ids = common_doc_ids[start:end]

        chunk_data = {
            "tfidf_chunk": tfidf_chunk,
            "bert_chunk": bert_chunk,
            "doc_ids": chunk_doc_ids
        }

        chunk_file = os.path.join(save_path, f"hybrid_chunk_{start}_{end}.joblib")
        joblib.dump(chunk_data, chunk_file, compress=3)

    print(f"✅ تم إنشاء {total_docs} وثيقة في دفعات داخل: {save_path}")




In [6]:
# 🔸 تنفيذ
build_clean_hybrid_representation_in_chunks(
    tfidf_path=r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Data Representation\TF-IDF\antique\train\doc\tfidf_data.joblib",
    bert_path=r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Data Representation\Bert\antique\train\doc\bert_embedding.joblib",
    save_path=r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Data Representation\Hybrid\antique\train\chunks",
    chunk_size=5000
)

🚀 بدء معالجة التمثيلات الهجينة مع التحقق من التداخل...
📌 عدد الوثائق المشتركة: 401768 / TF-IDF: 403666 / BERT: 401768


🧩 معالجة Chunks: 100%|██████████| 81/81 [01:09<00:00,  1.17it/s]


✅ تم إنشاء 401768 وثيقة في دفعات داخل: C:\Users\Azzam\PycharmProjects\PythonProject\Data Representation\Hybrid\antique\train\chunks


In [8]:
import os
import joblib
from sklearn.metrics.pairwise import cosine_similarity

def validate_hybrid_chunk_joblib(chunk_start, chunk_end, base_path, tfidf_dim=100000, bert_dim=384):
    # بناء اسم ملف chunk
    chunk_file = os.path.join(base_path, f"hybrid_chunk_{chunk_start}_{chunk_end}.joblib")

    if not os.path.exists(chunk_file):
        print(f"❌ الملف غير موجود: {chunk_file}")
        return

    print(f"\n📂 Checking hybrid chunk joblib {chunk_start}-{chunk_end}")

    # تحميل البيانات
    chunk_data = joblib.load(chunk_file)

    # التأكد من وجود المفاتيح المطلوبة
    required_keys = ["tfidf_chunk", "bert_chunk", "doc_ids"]
    for key in required_keys:
        if key not in chunk_data:
            print(f"❌ المفتاح '{key}' غير موجود في الملف")
            return

    tfidf_chunk = chunk_data["tfidf_chunk"]
    bert_chunk = chunk_data["bert_chunk"]
    doc_ids = chunk_data["doc_ids"]

    # التحقق من الأبعاد وعدد الوثائق
    num_docs = len(doc_ids)
    if tfidf_chunk.shape[0] != num_docs:
        print(f"❌ عدد الصفوف في tfidf_chunk ({tfidf_chunk.shape[0]}) لا يطابق عدد doc_ids ({num_docs})")
        return
    if bert_chunk.shape[0] != num_docs:
        print(f"❌ عدد الصفوف في bert_chunk ({bert_chunk.shape[0]}) لا يطابق عدد doc_ids ({num_docs})")
        return
    if tfidf_chunk.shape[1] != tfidf_dim:
        print(f"❌ عدد أعمدة tfidf_chunk غير متوقع: {tfidf_chunk.shape[1]} != {tfidf_dim}")
        return
    if bert_chunk.shape[1] != bert_dim:
        print(f"❌ عدد أعمدة bert_chunk غير متوقع: {bert_chunk.shape[1]} != {bert_dim}")
        return

    print(f"✅ Document count: {num_docs}")
    print(f"✅ TF-IDF chunk shape: {tfidf_chunk.shape}")
    print(f"✅ BERT chunk shape: {bert_chunk.shape}")

    # اختبار تشابه بسيط: نحسب تشابه cos بين أول وثيقتين باستخدام التمثيل الهجين (concat)
    if num_docs >= 2:
        # جمع التمثيل الهجين مؤقتًا (dense + sparse)
        from scipy.sparse import hstack

        hybrid_0 = np.hstack([tfidf_chunk[0].toarray().flatten(), bert_chunk[0]])
        hybrid_1 = np.hstack([tfidf_chunk[1].toarray().flatten(), bert_chunk[1]])

        sim = cosine_similarity([hybrid_0], [hybrid_1])[0][0]
        print(f"🧪 Cosine similarity بين أول وثيقتين في التمثيل الهجين: {sim:.4f}")
    else:
        print("ℹ️ Chunk يحتوي على وثيقة واحدة فقط، لا يمكن حساب التشابه.")

    print("✅ التمثيل الهجين في chunk صحيح.")


# مثال على الاستخدام
validate_hybrid_chunk_joblib(
    chunk_start=0,
    chunk_end=5000,
    base_path=r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Data Representation\Hybrid\beir\quora\test\chunks",
    tfidf_dim=102029,
    bert_dim=384
)




📂 Checking hybrid chunk joblib 0-5000
✅ Document count: 5000
✅ TF-IDF chunk shape: (5000, 102029)
✅ BERT chunk shape: (5000, 384)
🧪 Cosine similarity بين أول وثيقتين في التمثيل الهجين: 0.9058
✅ التمثيل الهجين في chunk صحيح.


In [9]:
import os
import joblib
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

def hybrid_parallel_search(
    query_tfidf_vector,
    query_bert_vector,
    chunks_path,
    tfidf_weight=0.5,
    bert_weight=0.5,
    top_k=10
):
    """
    بحث تمثيل متوازي مع دمج النتائج fusion weights

    - query_tfidf_vector: تمثيل الاستعلام باستخدام TF-IDF (sparse vector أو dense numpy array)
    - query_bert_vector: تمثيل الاستعلام باستخدام BERT (dense numpy array)
    - chunks_path: مجلد يحوي ملفات chunk بصيغة joblib مع تمثيلات TF-IDF و BERT
    - tfidf_weight, bert_weight: أوزان دمج التشابه (يجب أن مجموعهم=1)
    - top_k: عدد النتائج النهائية المراد إرجاعها
    """

    assert abs(tfidf_weight + bert_weight - 1.0) < 1e-6, "مجموع الأوزان يجب أن يساوي 1"

    final_results = []

    chunk_files = sorted([f for f in os.listdir(chunks_path) if f.endswith(".joblib")])
    for chunk_file in tqdm(chunk_files, desc="⚙️ معالجة chunks"):
        chunk_data = joblib.load(os.path.join(chunks_path, chunk_file))

        tfidf_chunk = chunk_data["tfidf_chunk"]  # sparse matrix
        bert_chunk = chunk_data["bert_chunk"]    # dense numpy array
        doc_ids = chunk_data["doc_ids"]

        # حساب تشابه TF-IDF (cosine similarity)
        sims_tfidf = cosine_similarity(query_tfidf_vector, tfidf_chunk).flatten()

        # حساب تشابه BERT (cosine similarity)
        sims_bert = cosine_similarity([query_bert_vector], bert_chunk).flatten()

        # دمج التشابهات بالأوزان المحددة
        sims_fused = tfidf_weight * sims_tfidf + bert_weight * sims_bert

        # جمع النتائج مع doc_ids
        final_results.extend(zip(doc_ids, sims_fused))

    # ترتيب النتائج النهائية تنازليًا حسب التشابه
    final_results.sort(key=lambda x: x[1], reverse=True)
    top_results = final_results[:top_k]

    print("\n✅ النتائج النهائية (Top {}):".format(top_k))
    for i, (doc_id, score) in enumerate(top_results, 1):
        print(f"{i}. {doc_id} | Score: {score:.4f}")

    return top_results


In [21]:
from sentence_transformers import SentenceTransformer
import joblib
import numpy as np

# 1. حضّر موديل BERT
bert_model = SentenceTransformer("all-MiniLM-L6-v2")

# 2. جهز الاستعلام
query = "how to learn a amchine programming language? "

# 3. حمّل vectorizer من ملف tfidf_data.joblib (لأنه يحتويه)
vectorizer_data = joblib.load(r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Data Representation\TF-IDF\beir\quora\test\doc\tfidf_data.joblib")
vectorizer = vectorizer_data["vectorizer"]

# 4. حول الاستعلام إلى تمثيل TF-IDF
query_tfidf_vector = vectorizer.transform([query])

# 5. تمثيل الاستعلام باستخدام BERT
query_bert_vector = bert_model.encode(query, convert_to_numpy=True).astype(np.float32)

# 6. نفّذ البحث (تأكد أن تابع hybrid_parallel_search معرف ومتاح)
results = hybrid_parallel_search(
    query_tfidf_vector=query_tfidf_vector,
    query_bert_vector=query_bert_vector,
    chunks_path=r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Data Representation\Hybrid\beir\quora\test\chunks",
    tfidf_weight=0.4,
    bert_weight=0.6,
    top_k=10
)

# 7. عرض النتائج
print(results)


⚙️ معالجة chunks: 100%|██████████| 105/105 [00:16<00:00,  6.55it/s]



✅ النتائج النهائية (Top 10):
1. 91642 | Score: 0.6005
2. 527808 | Score: 0.5991
3. 498101 | Score: 0.5965
4. 44161 | Score: 0.5810
5. 1271 | Score: 0.5602
6. 112819 | Score: 0.5481
7. 364397 | Score: 0.5396
8. 22650 | Score: 0.5242
9. 9811 | Score: 0.5242
10. 419330 | Score: 0.5234
[('91642', np.float64(0.6005096524953842)), ('527808', np.float64(0.5991446912288666)), ('498101', np.float64(0.5965242475271225)), ('44161', np.float64(0.5810033917427063)), ('1271', np.float64(0.5602440029382706)), ('112819', np.float64(0.5480646084057077)), ('364397', np.float64(0.5395946989761575)), ('22650', np.float64(0.524203137870819)), ('9811', np.float64(0.524203137870819)), ('419330', np.float64(0.5233781973825677))]
